### Exploration of IDynoMiCS SVG files to view data and create an animation of bacillis.xml output

In [4]:
from pathlib import Path
import re
from IPython.display import display, SVG

#Path to SVG files
svg_dir = Path("/Users/mayaabdalla/idyno_demo/runs/demo1/2025.08.28_14.09.26_764_bacilli/second_agents")

# Sort and get all SVG files
svg_files = sorted(svg_dir.glob("*.svg"))

#Confirm if all files were found (expect 240 files)
print(f"Found {len(svg_files)} SVG files")

# Extract timestep from filename (second_agents_00001_00001.svg)
def extract_time_from_filename(filename):
    match = re.search(r"_(\d+)_(\d+)\.svg$", filename.name)
    return int(match.group(2)) if match else -1

# Sort svg files by timestep
svg_files_sorted = sorted(svg_files, key=extract_time_from_filename)

# Display the first 2 for now
for svg_file in svg_files_sorted[:2]:
    print(f"Displaying: {svg_file.name}")
    display(SVG(filename=str(svg_file)))


Found 240 SVG files
Displaying: second_agents_00001_00001.svg


Displaying: second_agents_00002_00002.svg


# Cell Counts Animation (cell below) #

## Tried to animate the SVG and POV files as animation - had a hard time with configuration of cairo package...
###     - Had to include os line to tell point .venv to local libraries required
###     - Couldn't figure out for a longtime and still not fully working, so I asked CHATGPT for help on how to create SVG files or POV files into              an animation and suggested to make a .gif file
###     - File alone looks like snapshots BUT open in chrome and animation will work

In [1]:
#Trying to make a .gif animation of cell counts to see growth
# Need to point pythin to find libcairo, because .venv cant find it because its installed on macOS and not on .venv by homebrew
import os, subprocess

# Point to Homebrew install prefix (Apple Silicon = /opt/homebrew, Intel = /usr/local)
brew_prefix = subprocess.check_output(["brew", "--prefix"], text=True).strip()

# Tell Python’s dynamic loader where to find cairo and other libs
os.environ["DYLD_FALLBACK_LIBRARY_PATH"] = f"{brew_prefix}/lib"

# Also make sure brew’s bin is first in PATH (helps with subprocesses like rsvg-convert)
os.environ["PATH"] = f"{brew_prefix}/bin:" + os.environ["PATH"]

print("Brew prefix:", brew_prefix)
print("DYLD_FALLBACK_LIBRARY_PATH:", os.environ["DYLD_FALLBACK_LIBRARY_PATH"])


from pathlib import Path
import re
from PIL import Image
from cairosvg import svg2png

#SVG file path and path where to export animation
svg_movie_path = Path("/Users/mayaabdalla/idyno_demo/runs/demo1/2025.08.28_14.09.26_764_bacilli/second_agents")
output_gif_path = Path("second_agents_animation.gif")

#Image Size that may not have width/height defined
#some of the SVG files don't have these defined and need them to animate
default_size = (1000, 1000)

#Sort and extract timestep from the SVG filename
def extract_time_from_filename(filename):
    match = re.search(r"_(\d+)_(\d+)\.svg$", filename.name)
    return int(match.group(2)) if match else -1

#Get all SVG files and sort them in chronological order by timestep
svg_files = sorted(svg_movie_path.glob("*.svg"), key=extract_time_from_filename)


#Convert SVG to PNG and store in memory
frames = []
temp_pngs = []

for svg_file in svg_files:
    print(f"Processing: {svg_file.name}")
    png_path = svg_file.with_suffix(".png") #Same name as SVG file but with the PNG extension

    try:
        # Using CairoSVG to convert SVG to PNG
        # Set width and height in case the SVG file has no explicit size
        svg2png(
            url=str(svg_file),
            write_to=str(png_path),
            output_width=default_size[0],
            output_height=default_size[1]
        )
        img = Image.open(png_path).convert("RGB") # Open the PNG as a pillow image and convert to RBG for consistency

        # Store the image for GIF creation
        frames.append(img)
        temp_pngs.append(png_path)
    except Exception as e:
        print(f"Failed to convert {svg_file.name}: {e}")

# Save collected frames as an animated GIF
if frames:
    print(f"Saving GIF with {len(frames)} frames...")
    frames[0].save(
        output_gif_path,
        save_all=True, #save as animation
        append_images=frames[1:], #adding all frames after the first
        duration=150,  # milliseconds between frames
        loop=0
    )
    print(f" Saved animation to {output_gif_path.resolve()}")
else:
    print(" No frames were successfully generated.")


#The file is successful animation when you open the .gif in chrome


Brew prefix: /opt/homebrew
DYLD_FALLBACK_LIBRARY_PATH: /opt/homebrew/lib
Processing: second_agents_00001_00001.svg
Processing: second_agents_00002_00002.svg
Processing: second_agents_00003_00003.svg
Processing: second_agents_00004_00004.svg
Processing: second_agents_00005_00005.svg
Processing: second_agents_00006_00006.svg
Processing: second_agents_00007_00007.svg
Processing: second_agents_00008_00008.svg
Processing: second_agents_00009_00009.svg
Processing: second_agents_00010_00010.svg
Processing: second_agents_00011_00011.svg
Processing: second_agents_00012_00012.svg
Processing: second_agents_00013_00013.svg
Processing: second_agents_00014_00014.svg
Processing: second_agents_00015_00015.svg
Processing: second_agents_00016_00016.svg
Processing: second_agents_00017_00017.svg
Processing: second_agents_00018_00018.svg
Processing: second_agents_00019_00019.svg
Processing: second_agents_00020_00020.svg
Processing: second_agents_00021_00021.svg
Processing: second_agents_00022_00022.svg
Pro

## ecoli.XML file found - exploration below ##



java -jar ~/Downloads/idynomics/iDynoMiCS-2-July-2025/iDynoMiCS-2.0.jar \
-p ~/Downloads/idynomics/iDynoMiCS-2-July-2025/protocol/ecoli.xml \
-o ~/idyno_demo/runs/demo1

NOTE: This simulation time is 1,800,000 and timestep is 5 seconds takes 10 min for each timestep, the simulation was too long so stopped it at 91 frames, 10,801 step index

    real time t = (stepindex - 1) x Stepsize
              t = (10801 -1) x 5
              t = 54,000 seconds or 15 hours of simulation


File names are...biofilm_oxygen_CONCN_00004_00361.svg, biofilm_oxygen_CONCN_00005_00481.svg --> what is this 120 indexing?
- simulation step index is 1-based, tied to master timer
- SVG writer starts at 600s and repeats every 600s (10 min)
- step index is 1+ time/stepsize so:
        t = 600 --> 1 + 600/5 = 121
        t = 1800 --> 1 + 1800/5 = 361
        t = 2400 --> 1 + 2400/5 = 481
- the file names match the first index, so the 00004 is the frame, 361 and 481 is indexing



In [12]:
from pathlib import Path
import re
from IPython.display import display, SVG

#Path to SVG files
svg_dir = Path("/Users/mayaabdalla/Downloads/idynomics/iDynoMiCS-2-July-2025/results/2025.09.04_20.29.50_236_ecoli3D/biofilm_oxygen_CONCN")
# Sort and get all SVG files
svg_files = sorted(svg_dir.glob("*.svg"))

#Confirm if all files were found (expect 240 files)
print(f"Found {len(svg_files)} SVG files")

# Extract timestep from filename
# {5} is a match of 5 numbers
def extract_time_from_filename(filename):
    match = re.search(r"biofilm_oxygen_CONC_(\d{5})_(\d{5})\.svg$", filename.name)
    return int(match.group(2)) if match else -1

# Sort svg files by timestep
svg_files_sorted = sorted(svg_files, key=extract_time_from_filename)

# Display the first 2 for now
for svg_file in svg_files_sorted[:10]:
    print(f"Displaying: {svg_file.name}")
    display(SVG(filename=str(svg_file)))


Found 91 SVG files
Displaying: biofilm_oxygen_CONCN_00001_00001.svg


Displaying: biofilm_oxygen_CONCN_00002_00121.svg


Displaying: biofilm_oxygen_CONCN_00003_00241.svg


Displaying: biofilm_oxygen_CONCN_00004_00361.svg


Displaying: biofilm_oxygen_CONCN_00005_00481.svg


Displaying: biofilm_oxygen_CONCN_00006_00601.svg


Displaying: biofilm_oxygen_CONCN_00007_00721.svg


Displaying: biofilm_oxygen_CONCN_00008_00841.svg


Displaying: biofilm_oxygen_CONCN_00009_00961.svg


Displaying: biofilm_oxygen_CONCN_00010_01081.svg


 ### ecoli.xml animation (below) ###

In [21]:
#Trying to make a .gif animation of cell counts to see growth
# Need to point pythin to find libcairo, because .venv cant find it because its installed on macOS and not on .venv by homebrew
import os, subprocess
from pathlib import Path
import re
from PIL import Image
from cairosvg import svg2png

# Point to Homebrew install prefix (Apple Silicon = /opt/homebrew, Intel = /usr/local)
brew_prefix = subprocess.check_output(["brew", "--prefix"], text=True).strip()

# Tell Python’s dynamic loader where to find cairo and other libs
os.environ["DYLD_FALLBACK_LIBRARY_PATH"] = f"{brew_prefix}/lib"

# Also make sure brew’s bin is first in PATH (helps with subprocesses like rsvg-convert)
os.environ["PATH"] = f"{brew_prefix}/bin:" + os.environ["PATH"]


#Path to SVG files and configuration
svg_dir = Path("/Users/mayaabdalla/Downloads/idynomics/iDynoMiCS-2-July-2025/results/2025.09.04_20.29.50_236_ecoli3D/biofilm_oxygen_CONCN")
output_gif_path = Path("biofilm_oxygen_CONC.gif")
default_size = (1000, 1000)


# Extract timestep from filename
# {5} is a match of 5 numbers
def extract_time_from_filename(filename: str) -> int:
    match = re.search(r"biofilm_oxygen_CONCN_(\d{5})_(\d{5})\.svg$", filename)
    return int(match.group(2)) if match else -1

#Get all SVG files and sort them in chronological order by timestep
# Gather and sort SVGs by timestep index
svg_files = [p for p in svg_dir.glob("*.svg")]
print(f"Found {len(svg_files)} SVG files")

svg_files = sorted(svg_files, key=lambda p: extract_time_from_filename(p.name))

#Convert SVG to PNG and store in memory --> collect frames and make a .gif
frames = []
temp_pngs = []

for svg_path in svg_files:
    print(f"Processing: {svg_path.name}")
    png_path = svg_path.with_suffix(".png") #Same name as SVG file but with the PNG extension

    try:
        # Using CairoSVG to convert SVG to PNG
        # Set width and height in case the SVG file has no explicit size
        svg2png(
            url=str(svg_path),
            write_to=str(png_path),
            output_width=default_size[0],
            output_height=default_size[1],
        )
        img = Image.open(png_path).convert("RGB") # Open the PNG as a pillow image and convert to RBG for consistency
        # Store the image for GIF creation
        frames.append(img)
        temp_pngs.append(png_path)
    except Exception as e:
        print(f"Failed to convert {f}: {e}")

# Save collected frames as an animated GIF
if frames:
    print(f"Saving GIF with {len(frames)} frames...")
    frames[0].save(
        output_gif_path,
        save_all=True, #save as animation
        append_images=frames[1:], #adding all frames after the first
        duration=300,  # milliseconds between frames
        loop=0
    )
    print(f" Saved animation to {output_gif_path.resolve()}")
else:
    print(" No frames were successfully generated.")

Found 91 SVG files
Processing: biofilm_oxygen_CONCN_00001_00001.svg
Processing: biofilm_oxygen_CONCN_00002_00121.svg
Processing: biofilm_oxygen_CONCN_00003_00241.svg
Processing: biofilm_oxygen_CONCN_00004_00361.svg
Processing: biofilm_oxygen_CONCN_00005_00481.svg
Processing: biofilm_oxygen_CONCN_00006_00601.svg
Processing: biofilm_oxygen_CONCN_00007_00721.svg
Processing: biofilm_oxygen_CONCN_00008_00841.svg
Processing: biofilm_oxygen_CONCN_00009_00961.svg
Processing: biofilm_oxygen_CONCN_00010_01081.svg
Processing: biofilm_oxygen_CONCN_00011_01201.svg
Processing: biofilm_oxygen_CONCN_00012_01321.svg
Processing: biofilm_oxygen_CONCN_00013_01441.svg
Processing: biofilm_oxygen_CONCN_00014_01561.svg
Processing: biofilm_oxygen_CONCN_00015_01681.svg
Processing: biofilm_oxygen_CONCN_00016_01801.svg
Processing: biofilm_oxygen_CONCN_00017_01921.svg
Processing: biofilm_oxygen_CONCN_00018_02041.svg
Processing: biofilm_oxygen_CONCN_00019_02161.svg
Processing: biofilm_oxygen_CONCN_00020_02281.svg
P

# Biofilm.xml exploration #

### Realized my workflow is all over the place and needs organization... ###
 ####   - idyno_demo folder is now where the simulation output goes and idynomics folder is where the software calling and protocol files will be stored ####

### New code workflow
java -jar ~/Downloads/idynomics/iDynoMiCS-2-July-2025/iDynoMiCS-2.0.jar \
-p ~/Downloads/idynomics/iDynoMiCS-2-July-2025/protocol/biofilm.xml \
-o ~/idyno_demo/runs/demo2